This notebooks shows the performance evaluation of the EfficientNet-based model made by the Computational Systems Biology Laboratory. We first load some necessary libraries.

In [1]:
import os, random, gc
import re, time, json
import datetime
import glob
import pickle
import numpy as np
import pandas as pd 
import random

from PIL import Image
from matplotlib import pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve,roc_auc_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from IPython.display import Audio
from pathlib import Path
from sklearn import metrics

from  ast import literal_eval

from IPython.display import Audio
from sklearn.metrics import label_ranking_average_precision_score

import joblib

import timm
from pprint import pprint

import torch
from torch import nn, optim
from  torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import transforms

We then set the directory for our pretrained model.

In [16]:
torch.hub.set_dir("/project/dsc-is/malubay-a")  
model_names = timm.list_models(pretrained=True)

We then define our Enet class.

In [17]:
args = {}
model_name = 'efficientnet_b0'
data_config = timm.data.resolve_data_config({}, model=model_name, verbose=True)

class Enet(nn.Module):
    def __init__(self):
        super(Enet,self).__init__()
        self.model  = timm.create_model('efficientnet_b0', pretrained=True)
        self.model.classifier = nn.Sequential(
                                nn.Linear(1280, 512),
                                nn.ReLU(),
                                nn.Dropout(0.4),
                                nn.Linear(512, 128),
                                nn.ReLU(),
                                nn.Dropout(0.4),
                                nn.Linear(128, 2),
                                )

    def forward(self,x):
        x = self.model(x)
        return x

Loading our pretrained model

In [18]:
model_b0 = Enet().cuda()

saved_model = "ef0_sokuwan.pth"
if(os.path.exists(saved_model)):
    model_b0.load_state_dict(torch.load(saved_model))

Data Transformations and Utility Functions for Data Processing

In [19]:
data_transforms = {
    "train": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.RandomRotation(15),
        transforms.ToTensor()
    ]),
    "test": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    ,
    "own": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
}

class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data,label,transform = data_transforms['test']):
        self.transform = transform
        self.data = data
        self.label = label
 

    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image=Image.fromarray(np.uint16(image))
        image=image.convert("RGB")
        image=np.asarray(image)
        return image

    def __getitem__(self, idx):
        image = self.data[idx]
        image = self.normalize(image)
    

        if self.transform!=None:
            out_data = self.transform(image)
        
        else:
            out_data = self.transform(image)
        
        label = torch.tensor(self.label[idx]).float()

        return out_data,label

Loading the Dataset

In [6]:
dir_data = "/project/dsc-is/napi"

if not "data_all" in globals():
    tmp = np.load(os.path.join(dir_data, "dat0", "pix_w256_1.npz"))
    data1 = tmp['arr_0']
    print(data1.shape)
    tmp = np.load(os.path.join(dir_data, "dat0", "pix_w256_2.npz"))
    data2 = tmp['arr_0']
    print(data2.shape)
    
    data_all = np.concatenate((data1,data2), axis=0) 
    del data1
    del data2

print(data_all.shape)

(50297, 256, 256)
(32700, 256, 256)
(82997, 256, 256)


Getting the Diagnosis

In [7]:
diagnosis1 = pd.read_csv(os.path.join(dir_data, "dat0", "diagnosis_new.txt"), sep='\t', header=None)
yyy_scoliosis = (diagnosis1.iloc[:,4]=="有").to_numpy().astype(int)

We then use the model to predict on our dataset. Note that our model has two output units.

In [8]:
datasets = Mydatasets(data_all, yyy_scoliosis)
dataloader = DataLoader(datasets, batch_size=10, shuffle=False)

model_b0.eval()
out_all = []
for data, label in dataloader:
    with torch.no_grad():
        out_tmp = model_b0(data.cuda())
        out_all.append(out_tmp.cpu().numpy())
        
out_all = np.concatenate(out_all)

Since this model has two output units, we use softmax to get a "score" function which will be used to compute for the predicted labels and ROC-AUC score.

In [ ]:
def softmax(out):
    tmp = np.exp(out[:,0])+np.exp(out[:,1])
    return(np.exp(out[:,1]) / tmp)

In [ ]:
scores = softmax(out_all)
preds = scores >= 0.5
preds = preds.astype('int')

We now get the indices of true positives, false positives, false negatives, and true negatives.

In [9]:
idx_true_positive = np.where(np.logical_and(preds == 1, yyy_scoliosis == 1))[0]
idx_false_positive = np.where(np.logical_and(preds == 1, yyy_scoliosis == 0))[0]
idx_false_negative = np.where(np.logical_and(preds == 0, yyy_scoliosis == 1))[0]
idx_true_negative = np.where(np.logical_and(preds == 0, yyy_scoliosis == 0))[0]

We now construct a confusion matrix.

In [10]:
confusion_matrix = pd.DataFrame(columns = ['Positive Diagnosis', 'Negative Diagnosis', 'Totals'], 
                                index = ['Positive Prediction', 'Negative Prediction', 'Totals'],
                                data = [[idx_true_positive.shape[0], idx_false_positive.shape[0], idx_true_positive.shape[0] + idx_false_positive.shape[0]],
                                        [idx_false_negative.shape[0], idx_true_negative.shape[0], idx_false_negative.shape[0] + idx_true_negative.shape[0]],
                                        [idx_true_positive.shape[0] + idx_false_negative.shape[0], idx_false_positive.shape[0] + idx_true_negative.shape[0], 82997]])
confusion_matrix

,Positive Diagnosis,Negative Diagnosis,Totals
Positive Prediction,1235,3529,4764
Negative Prediction,73,78160,78233
Totals,1308,81689,82997


We now compute for some classification metrics. As seen below, our precision and recall values are quite low. This low precision value is compensated by a relatively high recall.

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

print(f'Accuracy: {round(100*accuracy_score(yyy_scoliosis, preds),3)}%')
print(f'Precision: {round(100*precision_score(yyy_scoliosis, preds),3)}%')
print(f'Recall: {round(100*recall_score(yyy_scoliosis, preds),3)}%')
print(f'F1-Score: {round(100*f1_score(yyy_scoliosis, preds),3)}%')
print(f'ROC_AUC: {round(100*roc_auc_score(yyy_scoliosis, scores),3)}%')

Accuracy: 95.66%
Precision: 25.924%
Recall: 94.419%
F1-Score: 40.679%
ROC_AUC: 98.725%
